In [49]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Préparation du jeu de données

In [50]:
import splitfolders
data_dir = 'dataset_Multiclass_Covid'

In [51]:
# Diviser les données en données train et test
splitfolders.ratio(data_dir, output='dataset', seed=555, ratio=(.8, .2))

Copying files: 532 files [00:00, 1011.77 files/s]


In [52]:
# générer les images : faible pourcentage d'images du dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [53]:
train_generator = train_datagen.flow_from_directory('dataset/train', target_size=(64, 64), batch_size=32, class_mode='categorical')

Found 424 images belonging to 4 classes.


In [54]:
test_generator = test_datagen.flow_from_directory('dataset/val', target_size=(64, 64), batch_size=32, class_mode='categorical')

Found 108 images belonging to 4 classes.


# Contruction du DEEP CNN

In [55]:
# initialisation du CNN
classifier = Sequential()

In [56]:
# première couche de convolution
classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
classifier.add(MaxPooling2D(pool_size=(2, 2)))


/home/ariel/PycharmProjects/enronmails/venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [57]:
# deuxième couche de convolution
classifier.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.2))

In [58]:
# troisième couche de convolution
classifier.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

In [59]:
# vectorisation
classifier.add(Flatten())
classifier.add(Dropout(0.3))

In [60]:
# ANN
classifier.add(Dense(512, activation='relu'))
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(4, activation='softmax'))

In [61]:
classifier.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,519,236 (9.61 MB)

 Trainable params: 2,519,236 (9.61 MB)

 Non-trainable params: 0 (0.00 B)

In [62]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [63]:
# création d'un model checkpoint
checkpointer = ModelCheckpoint(filepath='checkpoint/classifier.keras', monitor='accuracy', mode='max', verbose=1, save_best_only=True)

In [64]:
# implémentation pour l'arret precoce
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')

In [65]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

In [66]:
callbacks_list = [checkpointer, early_stopping, reduce_lr]

In [67]:
classifier.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,519,236 (9.61 MB)

 Trainable params: 2,519,236 (9.61 MB)

 Non-trainable params: 0 (0.00 B)

In [68]:
model = classifier.fit(train_generator, validation_data = test_generator, epochs = 20, verbose = 1, callbacks=callbacks_list)

/home/ariel/PycharmProjects/enronmails/venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step - accuracy: 0.2423 - loss: 1.4748
Epoch 1: accuracy improved from -inf to 0.23821, saving model to checkpoint/classifier.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 514ms/step - accuracy: 0.2420 - loss: 1.4720 - val_accuracy: 0.2963 - val_loss: 1.3780 - learning_rate: 0.0010
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step - accuracy: 0.2705 - loss: 1.3727
Epoch 2: accuracy improved from 0.23821 to 0.29481, saving model to checkpoint/classifier.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 442ms/step - accuracy: 0.2721 - loss: 1.3721 - val_accuracy: 0.4259 - val_loss: 1.3150 - learning_rate: 0.0010
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step - accuracy: 0.4285 - loss: 1.2582
Epoch 3: accuracy improved from 0.29481 to 0.45991, saving model to checkpoint/classifier.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 454ms/step - accuracy: 0.4305 - loss: 1.2536 - val_accuracy: 0.6296 - val_loss: 0.9486 - learning_rate: 0.0010
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━

In [69]:
train_generator.class_indices

{'Infection_Bacterienne': 0,
 'Infection_Covid': 1,
 'Infection_Virale': 2,
 'Normal': 3}

In [72]:
from sklearn.metrics import classification_report
y_pred_train = classifier.predict(test_generator)
y_predict = np.argmax(y_pred_train, axis=1)
print(classification_report(test_generator.classes, y_predict, target_names=test_generator.classes_))

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 279ms/step


AttributeError: 'DirectoryIterator' object has no attribute 'classes_'